In [1]:
import os
import nibabel as nib
import numpy as np
from tqdm import tqdm

In [2]:
def get_bounding_box(volume):
    """
    입력 3D 볼륨에서 비검은 영역의 최소/최대 인덱스를 계산합니다.
    """
    non_zero_coords = np.argwhere(volume > 0)  # 0이 아닌 값의 좌표 찾기
    min_coords = non_zero_coords.min(axis=0)  # 최소 좌표
    max_coords = non_zero_coords.max(axis=0)  # 최대 좌표
    return min_coords, max_coords

def calculate_common_bounding_box(folder_path):
    """
    주어진 폴더 내의 모든 NIfTI 파일에 대해 공통적으로 사용할 수 있는 최소 크기의 bounding box를 계산합니다.
    """
    min_coords = None
    max_coords = None

    nii_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.nii') or f.endswith('.nii.gz')]
    print(f"Found {len(nii_files)} NIfTI files in {folder_path}")

    for nii_file in tqdm(nii_files, desc="Processing NIfTI files"):
        # NIfTI 파일 로드
        nii_data = nib.load(nii_file).get_fdata()

        # Bounding Box 계산
        file_min, file_max = get_bounding_box(nii_data)

        if min_coords is None and max_coords is None:
            # 첫 번째 파일에서 초기화
            min_coords = file_min
            max_coords = file_max
        else:
            # 모든 파일에 대한 공통 영역 계산
            min_coords = np.maximum(min_coords, file_min)  # 최소 좌표를 더 큰 값으로 업데이트
            max_coords = np.minimum(max_coords, file_max)  # 최대 좌표를 더 작은 값으로 업데이트

    # 최종 Bounding Box 반환
    return min_coords, max_coords

def crop_and_save(folder_path, output_folder, min_coords, max_coords):
    """
    공통 bounding box를 기준으로 모든 NIfTI 파일을 잘라낸 뒤 저장합니다.
    """
    os.makedirs(output_folder, exist_ok=True)

    nii_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.nii') or f.endswith('.nii.gz')]
    for nii_file in tqdm(nii_files, desc="Cropping and saving NIfTI files"):
        # NIfTI 파일 로드
        nii = nib.load(nii_file)
        nii_data = nii.get_fdata()

        # 데이터 크롭
        cropped_data = nii_data[
            min_coords[0]:max_coords[0]+1,
            min_coords[1]:max_coords[1]+1,
            min_coords[2]:max_coords[2]+1
        ]

        # 저장
        cropped_nii = nib.Nifti1Image(cropped_data, nii.affine, nii.header)
        output_path = os.path.join(output_folder, os.path.basename(nii_file))
        nib.save(cropped_nii, output_path)

# 사용 예시
folder_path = "niis/ADNI3"  # NIfTI 파일들이 있는 폴더 경로
output_folder = "niis_cropped/ADNI#"  # 잘라낸 결과를 저장할 폴더 경로

# 1. 공통 Bounding Box 계산
min_coords, max_coords = calculate_common_bounding_box(folder_path)
print(f"Common bounding box: min={min_coords}, max={max_coords}")

# 2. 크롭 및 저장
crop_and_save(folder_path, output_folder, min_coords, max_coords)
print("Cropping and saving completed!")

Found 2202 NIfTI files in niis/ADNI3


Processing NIfTI files: 100%|██████████| 2202/2202 [20:02<00:00,  1.83it/s]


Common bounding box: min=[55 23 18], max=[226 230 149]


Cropping and saving NIfTI files: 100%|██████████| 2202/2202 [12:51<00:00,  2.86it/s]

Cropping and saving completed!
